# 遇到前置按鈕


## 何謂Cookie

假如你登入了→伺服器會給你一組cookies
下次打開頁面時用header傳送cookies→伺服器認得你→跳過登入

In [3]:
import requests
requests.Session() #將request所收到的cookie存起來

# 範例1
以下範例為PTT八卦版
https://www.ptt.cc/ask/over18?from=%2Fbbs%2FGossiping%2Findex.html

查看點選"已滿18歲"後，所送出的封包

![title](img/post2.png)

In [5]:
import requests
r = requests.Session()
payload ={
    "from":"/bbs/Gossiping/index.html",
    "yes":"yes"
}
r1 = r.post("https://www.ptt.cc/ask/over18?from=%2Fbbs%2FGossiping%2Findex.html",payload) #模仿已滿18歲按鈕以post送出
r2 = r.get("https://www.ptt.cc/bbs/Gossiping/index.html")#爬主頁面
print(r2.text)

<!DOCTYPE html>
<html>
	<head>
		<meta charset="utf-8">
		

<meta name="viewport" content="width=device-width, initial-scale=1">

<title>看板 Gossiping 文章列表 - 批踢踢實業坊</title>

<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.25/bbs-common.css">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.25/bbs-base.css" media="screen">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.25/bbs-custom.css">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.25/pushstream.css" media="screen">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.25/bbs-print.css" media="print">




	</head>
    <body>
		
<div id="topbar-container">
	<div id="topbar" class="bbs-content">
		<a id="logo" href="/bbs/">批踢踢實業坊</a>
		<span>&rsaquo;</span>
		<a class="board" href="/bbs/Gossiping/index.html"><span class="board-label">看板 </span>Gossiping</a>
		<a class="right small" href="/about.html">關於我們</a>
		<a class="right small" href="/co

## 結合cookie爬下標題

In [8]:
import requests
from bs4 import BeautifulSoup
r = requests.Session()
payload ={
    "from":"/bbs/Gossiping/index.html",
    "yes":"yes"
}
r1 = r.post("https://www.ptt.cc/ask/over18?from=%2Fbbs%2FGossiping%2Findex.html",payload) #模仿已滿18歲按鈕以post送出
r2 = r.get("https://www.ptt.cc/bbs/Gossiping/index.html")#爬主頁面
soup = BeautifulSoup(r2.text, "html.parser")
sel = soup.select("div.title a")
for s in sel:
    print(s.text)

Re: [新聞] 恐怖份子愛用豐田車 溫朗東：Toyota需負
[新聞] 中科院研發海龍王系統 國艦國造擬採用
[問卦] hccc之後要幹嘛（綠色狸貓陣線文）
Re: [問卦] 我國綜藝節目是不是慘到不能再慘了
[新聞] 鍵盤白色恐怖逼噤聲？ 蘇上豪：不會屈服
[新聞] 相信葛特曼卻遭網軍噤聲?蘇上豪:此時台灣
[問卦] 有沒有寄生蟲的八卦???
Re: [問卦] 你們有女友嗎？
[問卦] 陣頭小孩平日行程是什麼呢?
[問卦] 桃園哪裡好玩？急
[問卦] 沖繩~長灘島~沙巴~峇里島 哪一個好玩又CP值高?
[問卦] 如果台男賣冰跟土耳其人一樣亂甩 ●"＿"
[問卦] 如何融入臺北車站？
[新聞] 國慶預演4次 機179航班延誤、估5萬旅客受
[新聞] 決定為兒站台 扁：有被抓回去的打算
[問卦] 有健身的484一晚睡懶叫就無法控制
[問卦] 肥宅晚上會唱什麼歌？
[新聞] 質疑柯文哲還債治水能力 姚文智：liar
Re: [問卦] 北漂真的是大環境不好,還是自己不努力?
[問卦] 金鐘為什麼不設政論節目獎？
[公告] 八卦板板規(2017.11.11)
[公告] 政問罰則暫時提高至16個月
[公告] 十月置底閒聊聊天文
[協尋] 107年9月29日三重區三和路4段20號車禍
[協尋] 10/04 20:30三峽學府路行車紀錄畫面


# 範例2
以下範例以逢甲大學打卡系統
https://signin.fcu.edu.tw/clockin/login.aspx

![title](img/打卡1.png)

藉由輸入帳號密碼送出post，觀察post封包內的參數
![title](img/打卡2.png)

得知參數有
loginData = {EVENTTARGET,EVENTARGUMENT,VIEWSTATE,VIEWSTATEGENERATOR,EVENTVALIDATION,LoginLdap\$UserName,LoginLdap\$Password,LoginLdap\$LoginButton}

較特別的地方於 EVENTVALIDATION 和 VIEWSTATE 和 VIEWSTATEGENERATOR
但是每次重整後值會不一樣，那就只能從程式碼找找看哪邊每次賦予值的

![title](img/打卡3.png)
![title](img/打卡4.png)

In [9]:
import requests
from bs4 import BeautifulSoup
s = requests.Session()
loginPage = s.get("https://signin.fcu.edu.tw/clockin/login.aspx")
bs = BeautifulSoup(loginPage.text, "html.parser")
loginData = {
    "__EVENTTARGET":"",
    "__EVENTARGUMENT":"",
    "__VIEWSTATE":bs.select("#__VIEWSTATE")[0]["value"],
    "__VIEWSTATEGENERATOR":bs.select("#__VIEWSTATEGENERATOR")[0]["value"],
    "__EVENTVALIDATION":bs.select("#__EVENTVALIDATION")[0]["value"],
    "LoginLdap$UserName":"學號",
    "LoginLdap$Password":"密碼",
    "LoginLdap$LoginButton":"登入"
}

In [12]:
import requests
from bs4 import BeautifulSoup
s = requests.Session()
loginPage = s.get("https://signin.fcu.edu.tw/clockin/login.aspx")
bs = BeautifulSoup(loginPage.text, "html.parser")
loginData = {
    "__EVENTTARGET":"",
    "__EVENTARGUMENT":"",
    "__VIEWSTATE":bs.select("#__VIEWSTATE")[0]["value"],
    "__VIEWSTATEGENERATOR":bs.select("#__VIEWSTATEGENERATOR")[0]["value"],
    "__EVENTVALIDATION":bs.select("#__EVENTVALIDATION")[0]["value"],
    "LoginLdap$UserName":"D0449733",
    "LoginLdap$Password":"As860117",
    "LoginLdap$LoginButton":"登入"
}
s.headers.update({"Referer":"https://signin.fcu.edu.tw/clockin/login.aspx"}) #比對瀏覽器與程式碼送出的header是否有差別，嘗試補足差別
s.post("https://signin.fcu.edu.tw/clockin/login.aspx", data=loginData)
login = s.get("https://signin.fcu.edu.tw/clockin/Student.aspx")
print(login.text)



<!DOCTYPE html>

<html xmlns="http://www.w3.org/1999/xhtml">
<head><title>
	學生簽到
</title><meta name="viewport" content="width=device-width, initial-scale=1.0" /><link rel="shortcut icon" type="image/x-icon" href="img/favicon.ico" /><link href="Content/bootstrap.min.css" rel="stylesheet" /><link href="Content/bootstrap-theme.css" rel="stylesheet" />
    <script src="Scripts/jquery-2.1.4.min.js"></script>
    <script src="Scripts/bootstrap.min.js"></script>
    <style type="text/css">
        .btn-margin {
            margin: 6px;
            width: 250px;
        }
        .btn-menu {
            border-width:1px;
            border-radius:3px;
            border-color:gray;
            background-color:rgb(240,239,245);
            background-image:none;
            width:300px;
            height:80px;
        }
    </style>
    <script>
        (function (i, s, o, g, r, a, m) {
            i['GoogleAnalyticsObject'] = r; i[r] = i[r] || function () {
                (i[r].q = i[r].q

# 範例3
以下範例以房仲網站 https://buy.yungching.com.tw/region

![title](img/房仲1.png)

In [18]:
#先嘗試使用基本爬蟲
import requests
from bs4 import BeautifulSoup
r = requests.get("http://buy.yungching.com.tw/region")
soup = BeautifulSoup(r.text,"html.parser")
sel = soup.select("a.item-title h3")
for s in sel:
    print(s.text)

東華店面美透天　台北市北投區東華街二段
政大附中旁大四房　台北市文山區政大二街
小家庭首購首選　台北市中山區長春路
雅緻電梯美廈三房　台北市信義區虎林街
運動公園精美透天　台北市士林區芝玉路二段
超值松江捷運小宅　台北市中山區松江路
石牌漂亮昇陽麗石　台北市北投區裕民二路
捷運邊間方正三樓　台北市北投區中央北路四段
捷運電梯景觀美屋　台北市內湖區成功路四段
展覽館捷運一樓　台北市南港區富康街
石牌全新麗緹車位　台北市北投區裕民六路
敦南101景觀宅　台北市大安區通安街
忠孝頂好四房坡車　台北市大安區建國南路一段
朝南庭園文教店辦　台北市松山區民生東路四段
國泰方正三房美寓　台北市中正區三元街
最美觀星窗景三房　台北市文山區木柵路二段
雋藏庭院裝潢大戶　台北市文山區政大一街
運動公園庭院泳池　台北市士林區芝玉路二段
近捷運美妝二樓　台北市北投區立農街一段
政大附中設籍美屋　台北市文山區政大一街
